In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [3]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
# import pybpr
import matplotlib.pyplot as plt
from functools import partial
from sklearn.feature_extraction import DictVectorizer
from scipy.special import expit

In [4]:
from pybpr import *

In [5]:
ui = UserItemData(
    n_users=100, 
    n_items =4,
    n_user_features=100,
    n_item_features=2
)
ui.add_interactions(
    user_indices=list(range(ui.n_users)), 
    item_indices=[0 for _ in range(ui.n_users)],
    weights=np.ones(ui.n_users)
)
ui.add_user_features(
    user_indices=list(range(ui.n_users)), 
    feature_indices=list(range(ui.n_users))
)
ui.add_item_features(
    item_indices=[0,1,2,3], 
    feature_indices=[0,0,1,1]
)
# # ui_data.add_item_features(item_indices=[10,20],feature_indices=[20,0])
# # rs=RecSys(
# #     ui_data=ui_data,
# #     n_latent=10
# # )
ui

UserItemData(
  n_users=100
  n_items=4
  n_interactions=100
  n_user_features=100
  n_user_feature_data=100
  n_item_features=2
  n_item_feature_data=4
)

In [6]:
optimizer = partial(
    torch.optim.SGD,
    lr=0.1,
    weight_decay=0.01
)
rs = RecSys(
    ui_data = ui,
    n_latent=2,
    optimizer=optimizer
)
rs

RecSys: Initiating Hybrid Recomender System..
RecSys: Got 100/100 +ve interactions
RecSys: Got 0/100 -ve interactions


RecSys(
data=UserItemData(
  n_users=100
  n_items=4
  n_interactions=100
  n_user_features=100
  n_user_feature_data=100
  n_item_features=2
  n_item_feature_data=4
)
model=HybridMF(
  (user_latent): Embedding(100, 2)
  (item_latent): Embedding(2, 2)
  (item_biases): Embedding(2, 1)
)
optimizer=SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.01
)
)

In [7]:
rs.fit(n_iter=1000,batch_size=100)

RecSys: Setting rng seed to 1234
RecSys: # pos interactions = 100
RecSys: Batch size(100)>data count(100). Setting batch size=100.
RecSys: Batch size = 100
RecSys: # of minibatches = 1
HybBPR:   0%|          | 1/1000 [00:00<02:16,  7.34it/s]

/kfs2/projects/zazzle/pybpr/pybpr/utils.py:25: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


HybBPR: 100%|██████████| 1000/1000 [00:02<00:00, 419.23it/s]


In [34]:
Fu = csr_scipy_to_torch(rs.Fu_csr)
Fi = csr_scipy_to_torch(rs.Fi_csr)
Eu = rs.model.user_latent.weight
Ei = rs.model.item_latent.weight
Bi = rs.model.item_biases.weight

xx = torch.sparse.mm(Fu,Eu)
yy = torch.sparse.mm(Fi,Ei)
yy2 = torch.sparse.mm(Fi,Bi)
zz = xx @ yy.T + yy2.T
zz

tensor([[ 1.6901,  1.6901, -1.7542, -1.7542],
        [ 1.8430,  1.8430, -1.7794, -1.7794],
        [ 1.7476,  1.7476, -1.7525, -1.7525],
        [ 1.7852,  1.7852, -1.7934, -1.7934],
        [ 1.7630,  1.7630, -1.7791, -1.7791],
        [ 1.7883,  1.7883, -1.7735, -1.7735],
        [ 1.7870,  1.7870, -1.7814, -1.7814],
        [ 1.8421,  1.8421, -1.7890, -1.7890],
        [ 1.8243,  1.8243, -1.7792, -1.7792],
        [ 1.8464,  1.8464, -1.8088, -1.8088],
        [ 1.7950,  1.7950, -1.7593, -1.7593],
        [ 1.8082,  1.8082, -1.7953, -1.7953],
        [ 1.8436,  1.8436, -1.7897, -1.7897],
        [ 1.8249,  1.8249, -1.8038, -1.8038],
        [ 1.8235,  1.8235, -1.7775, -1.7775],
        [ 1.8602,  1.8602, -1.7916, -1.7916],
        [ 1.7237,  1.7237, -1.7590, -1.7590],
        [ 1.8255,  1.8255, -1.8061, -1.8061],
        [ 1.8080,  1.8080, -1.7826, -1.7826],
        [ 1.8416,  1.8416, -1.8055, -1.8055],
        [ 1.8193,  1.8193, -1.7917, -1.7917],
        [ 1.8066,  1.8066, -1.7633

In [ ]:
rs.Fu

In [36]:
rs.model.item_latent.weight

Parameter containing:
tensor([[-0.1849,  0.2607],
        [-0.0043, -0.0999]], requires_grad=True)

In [30]:
Bi

Parameter containing:
tensor([[ 1.7837],
        [-1.7837]], requires_grad=True)

In [ ]:
torch.sparse.mm(Fi,Bi).shape

In [ ]:
Bi.shape, Fi.shape

In [ ]:
p_u = rs.model.user_latent(uid)
p_u = p_u*uw[..., None]
p_u = p_u.sum(dim=1)

In [ ]:
q_all = rs.model.item_latent.weight
p_u.shape, q_all.shape

In [ ]:
xx = np.random.RandomState()
xx.set_state(123)

In [ ]:
coo = rs.data.Fu_coo
values = coo.data
indices = np.vstack((coo.row, coo.col))
values.shape, indices.shape

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = coo.shape

xx = torch.sparse_coo_tensor(indices=i, values=v, size=torch.Size(shape))
xx

In [ ]:
pos_user_ids = [1,3,4]
pos_item_ids = [0,1,2]

Fu_sliced = csr_scipy_to_torch(rs.Fu_csr[pos_user_ids])
Fi_sliced_pos = csr_scipy_to_torch(rs.Fi_csr[pos_item_ids])


In [ ]:
Pu, Qi, Bu, Bi = rs.model.forward(Fu_sliced, Fi_sliced_pos)

In [ ]:
torch.mul(Pu, Qi).sum(dim=-1).shape

In [ ]:
Bu.squeeze().shape

In [ ]:
yy = rs.data.Fu_coo.tocsr()[[2,3]]
yy.todense()

In [ ]:
yy.shape

In [ ]:
zz = torch.sparse_csr_tensor(
    torch.LongTensor(yy.indptr),
    torch.LongTensor(yy.indices),
    torch.FloatTensor(yy.data),
    size=torch.Size(yy.shape)
)
zz

In [ ]:
zz

In [ ]:
yy = rs.model.user_latent.weight
zz = torch.sparse.mm(xx,yy)

In [ ]:
zz.shape

In [ ]:
for i in rs.data.Fi_coo.todok().items():
    print(i)

In [ ]:
for i in rs.data.Fi_coo.todok()[[0,3]].items():
    print(i)

In [ ]:
#initiate
nlatent = 2
user_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(user_df.shape[1],nlatent))
item_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(item_df.shape[1],nlatent))
print(item_latent_mat.shape, user_latent_mat.shape)
np.round(item_latent_mat,2)

In [ ]:
item_df.values[:, item_df.shape[0]:]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=nlatent).fit(item_df.values[:, item_df.shape[0]:])
pca_vecs = np.round(pca.components_,3)
pca_vecs.T

In [ ]:
np.round(np.dot(pca_vecs[0,:], pca_vecs[1,:]),2)

In [ ]:
item_latent_mat[item_df.shape[0]:,:] = pca.components_.T
np.round(item_latent_mat,2)

In [ ]:
np.round(np.linalg.norm(item_latent_mat, axis=1),2)

In [ ]:
for ix in range(n_items):
    item_latent_mat[ix,:] /= 100*np.linalg.norm(item_latent_mat[ix,:])
for ix in range(n_features):
    item_latent_mat[n_items+ix,:] /= np.linalg.norm(item_latent_mat[n_items+ix,:])
for ix in range(user_latent_mat.shape[0]):
    user_latent_mat[ix,:] /= 10*np.linalg.norm(user_latent_mat[ix,:])
np.round(np.linalg.norm(item_latent_mat, axis=1),2)
#np.round(np.linalg.norm(user_latent_mat, axis=1),2)

In [ ]:
np.round(item_latent_mat,2)

In [ ]:
np.round(user_latent_mat,2)

In [ ]:
for user_u in range(n_users):
    print('Recs for user: ', user_u)
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))

In [ ]:
# interaction
user_u = 0
item_pos = 0
item_neg = 1

In [ ]:
#gradients
for ix in range(10): 
    print(f'::::::::Iter{ix}')
    # slice user-item latent mat
    user_latent = user_latent_mat[user_u]
    #print(np.round(user_latent,2))
    pos_inds = item_df.iloc[item_pos].values.nonzero()[0]
    item_latent_pos = item_latent_mat[pos_inds,:]
    #print(np.round(item_latent_pos,2))
    item_latent_pos = np.sum(item_latent_pos,axis=0)
    #print(np.round(item_latent_pos,2))
    neg_inds = item_df.iloc[item_neg].values.nonzero()[0]
    item_latent_neg = item_latent_mat[neg_inds,:]
    #print(np.round(item_latent_neg,2))
    item_latent_neg = np.sum(item_latent_neg, axis=0)
    #print(np.round(item_latent_neg,2))

    r_uij = np.dot(user_latent, (item_latent_pos - item_latent_neg))
    sigmoid = expit(-r_uij)
    sigmoid_tiled = np.tile(sigmoid, (nlatent,))
    grad_u = np.multiply(sigmoid_tiled, (item_latent_neg - item_latent_pos))
    grad_pos = np.multiply(sigmoid_tiled, -user_latent)
    grad_neg = np.multiply(sigmoid_tiled, user_latent)

    # bpr update
    learning_rate=0.1
    user_latent_mat[user_u] -= learning_rate * grad_u
    item_latent_mat[item_pos] -= learning_rate * grad_pos
    item_latent_mat[item_neg] -= learning_rate * grad_neg
    print('User:\n', np.round(user_latent_mat,3))
    print('Item:\n', np.round(item_latent_mat,3))
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))
    #print('Recs:', np.round(rec,3))